# Простая сверточная нейронная сеть для MNIST

Пример простой CNN для MNIST, которая демонстрирует сверточные уровни, уровни объединения и уровни исключения.

In [1]:
# Simple CNN for the MNIST Dataset
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils

Загружаем набор данных MNIST и изменяем его форму, чтобы он подходил для использования при обучении CNN. В Keras слои, используемые для двумерных сверток, ожидают значения пикселей с размерами [пиксели] [ширина] [высота] [каналы].

В этом примере принудительно применяется так называемый порядок каналов по последнему каналу для единообразия.

В случае RGB последние пиксели измерения будут равны 3 для красного, зеленого и синего компонентов, и это будет похоже на наличие 3 входов изображения для каждого цветного изображения. В случае MNIST, где значения пикселей представляют собой шкалу серого, размер пикселя устанавливается равным 1.

In [2]:
# load data
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# reshape to be [samples][width][height][channels]
X_train = X_train.reshape((X_train.shape[0], 28, 28, 1)).astype('float32')
X_test = X_test.reshape((X_test.shape[0], 28, 28, 1)).astype('float32')

Нормализация значения пикселей в диапазоне от 0 до 1 и горячее кодирование выходных переменных.

In [3]:
# normalize inputs from 0-255 to 0-1
X_train = X_train / 255
X_test = X_test / 255

# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

Сетевая архитектура.

- 1 - Скрытый слой - это сверточный слой, называемый Convolution2D. Слой имеет 32 карты характеристик размером 5 × 5 и функцию активации выпрямителя. Это входной слой, ожидающий изображений с контуром структуры [пикселей] [ширина] [высота].

- 2 - Уровень пула, который принимает максимум, который называется MaxPooling2D. Он сконфигурирован с размером пула 2 × 2.

- 3 - Уровень регуляризации, использующий выпадение, называемый Dropout. Он настроен на случайное исключение 20% нейронов в слое, чтобы уменьшить переобучение.

- 4 - Далее идет слой, который преобразует данные 2D-матрицы в вектор, называемый Flatten. Это позволяет обрабатывать выходные данные стандартными полносвязными слоями.

- 5 - Полно-связанный слой со 128 нейронами и функцией активации выпрямителя.

- 6 - Выходной слой имеет 10 нейронов для 10 классов и функцию активации softmax для вывода вероятностных прогнозов для каждого класса.

Модель обучается с использованием логарифмических потерь loss='categorical_crossentropy' и алгоритма градиентного спуска ADAM.

In [4]:
# define a simple CNN model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Conv2D(32, (5, 5), input_shape=(28, 28, 1), activation='relu'))
    model.add(MaxPooling2D())
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

Оценивается модель многослойным персептроном. Рассчитана на 10 эпох с обновлением каждые 200 изображений.

In [5]:
# build the model
model = baseline_model()
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("CNN Error: %.2f%%" % (100-scores[1]*100))

Epoch 1/10
300/300 [==============================] - 11s 37ms/step - loss: 0.2381 - accuracy: 0.9309 - val_loss: 0.0748 - val_accuracy: 0.9773
Epoch 2/10
300/300 [==============================] - 11s 36ms/step - loss: 0.0733 - accuracy: 0.9781 - val_loss: 0.0540 - val_accuracy: 0.9828
Epoch 3/10
300/300 [==============================] - 11s 36ms/step - loss: 0.0533 - accuracy: 0.9835 - val_loss: 0.0442 - val_accuracy: 0.9856
Epoch 4/10
300/300 [==============================] - 11s 36ms/step - loss: 0.0417 - accuracy: 0.9872 - val_loss: 0.0376 - val_accuracy: 0.9880
Epoch 5/10
300/300 [==============================] - 11s 36ms/step - loss: 0.0359 - accuracy: 0.9891 - val_loss: 0.0365 - val_accuracy: 0.9867
Epoch 6/10
300/300 [==============================] - 11s 37ms/step - loss: 0.0279 - accuracy: 0.9912 - val_loss: 0.0322 - val_accuracy: 0.9887
Epoch 7/10
300/300 [==============================] - 11s 36ms/step - loss: 0.0236 - accuracy: 0.9923 - val_loss: 0.0355 - val_accuracy: